# Exercise UD1.2.3 Anharmonic Baths

**Compute the norm of $T_2$ as a function of the dissipative rate of the harmonic oscillator** $\kappa$.


The code provided simulates the master equation of a two-level system coupled to a dissipative harmonic oscillator in contact with a bath at temperature $T$

The evolution / dynamics of the total system can be described as
$$\frac{d\rho}{dt}=L\rho$$
with 
$$L\rho=-i[H,\rho]+\kappa(n_{th}+1)(2a\rho a^\dagger-a^\dagger a \rho-\rho a^\dagger a)+\kappa(n_{th})(2a^\dagger\rho a-a a^\dagger \rho-\rho a a^\dagger)$$
where $\kappa$ is the rate of dissipation, $n_{th}$ is the thermal occupation of the oscillator due to the temperature of the environment and 
$$H=\frac{\omega_a}{2}\sigma_z+\nu a^\dagger a+g\sigma_x (a^\dagger + a)$$
is the Hamiltonian, with $\omega_a$ the transition frequency of the two-level system (it's term corresponds to the hamiltonian of the 2LS, without constant terms), $\nu$ the frequency of the oscillator (it's term the hamiltonian of the oscillator) and $g$ the coupling strength between both.

Obviously all terms must act on the same hilbert space (the total hilbert space composed by the tensor product of the hilbert space of the 2LS and the one of the oscillator), so in this hamiltonian $\sigma_z$ in fact represents $\sigma_z \otimes I_v$ an similar to the other terms (we will use the subindex a to refer to the atom and v to the vibrational modes / oscillator).

In [ ]:
# So, let's start by writing (as is done in the code provided) the Hamiltonian and Liouvillian of the explained situation
# A two-level system (2LS, that we denote with 'a' of atom) coupled to an anharmonic oscillator (AO, denoted with 'v' of vibrations) that is in contact with a thermal bath.

import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt

# time parameters
dt=0.2
tmax=1000
# system, bath and coupling parameters
Delta=0.5   # detuning of 2LS
Omega=0     # Rabi frequency of 2LS
Gamma=0.5   # decay rate of 2LS
nu=1        # frequency of oscillator
kappa=0.01  # decay rate of oscillator
nbar=0      # mean thermal occupation number of bath (now its a zero-temperature bath, that's why we get the final ground state for the oscillator and the excited state for the 2LS)
g=.1        # coupling strength between 2LS and oscillator

nmax=10     # truncation of oscillator Hilbert space (if the cuttoff is too small the results will be wrong)
# ideally nmax should be several times larger than nbar (and nmax>>1, ideally infinite but this is not possible numerically)

# define sigma z
sz= np.array([[1,0],
             [0,-1]])
# define sigma x
sx= np.array([[0,1],
             [1,0]])
# define sigma plus
sp= np.array([[0,1],
             [0,0]])
# define sigma minus
sm= np.array([[0,0],
             [1,0]])
# define number operator
n=np.diag(np.arange(nmax))
# define annihilation and creation operators for oscillator
a=np.diag(np.sqrt(np.arange(nmax-1)+1),1)
ad=a.T.conj()
# define identity for oscillator
Idv=np.eye(nmax)
# define identity for 2LS
Ida = np.eye(2)

# define Hamiltonians
Hv=nu*n # oscillator Hamiltonian (v of vibrational modes)
Ha = Delta*sz + Omega*sx # 2LS Hamiltonian (a of atom)
# total Hamiltonian
H=np.kron(Ha,Idv)+np.kron(Ida,Hv)+g*np.kron(sx,ad+a) # the position of the oscillator is proportional to (a+ad)

# define total identity (of the full hilbert space)
Id=np.kron(Ida,Idv)
# define total creation and annihilation operators
Ad=np.kron(Ida,ad)
A=np.kron(Ida,a)
# define total number operator
N=Ad@A
# define total sigma z operator
SZ=np.kron(sz,Idv)
# we use capitals to denote operators acting on the full Hilbert space (the 'promoted' operators)

# construct Liouvillian
L=(1j*(np.kron(H,Id)-np.kron(Id,H.T))+ # Hamiltonian part (the commutator beetween H and rho)
   kappa*(nbar)*(2*np.kron(Ad,A.T)-np.kron(A@Ad,Id)-np.kron(Id,A@Ad))+ # Dissipator part due to absorption (gets population up)
   kappa*(nbar+1)*(2*np.kron(A,Ad.T)-np.kron(Ad@A,Id)-np.kron(Id,Ad@A))) # Dissipator part due to emission (gets population down)
# the nbar and nbar+1 terms correspond to the thermal bath effects, without them it would be an infinite-temperature bath
# the relation between nbar and the temperature is nbar/(nbar+1)=exp(-hbar*nu/kB*T)  (kB is the Boltzmann constant, we call exp(-hbar*nu/kB*T) the Boltzmann factor)


With the help of the Expander and Tracer superoperators, compute the second transfer tensor of the two-level system
$$T_2=E_2-E_1E_1$$
where $E_k=Tr \;e^{kdtL}Ex$ is the dynamical map propagating density matrices of the two-level system from $t=0$ to $t=kdt$.

(Ek=Tracer@EL@Expander, with EL=expm(k*L*dt))

In [ ]:
# So, let's exponentiate the Liouvillian to get the dynamical map and define the initial state
# As we want to compute two-time correlation functions, we also exponentiate the Liouvillian for a time 2*dt
# This will be useful later to compute the tensor correlation functions
# exponentiate Liouvillian, dymanical map
EL=expm(L*dt)
EL2=expm(L*2*dt)

# initial state
rhov0=np.diag(np.zeros(nmax)) # oscillator initially in ground state
rhov0[0,0]=1
rhoa0 = np.array([[1,0],   # 2LS initially in excited state
             [0,0]])
rho0=np.kron(rhoa0,rhov0)
rhot=rho0.flatten()

In [ ]:
# Now, we define the superoperators Tracer and Expander to compute the the two-time correlation functions (T2, tensor correlation matrix)
# Tracer does the partial trace over the oscillator degrees of freedom
# Expander expands the 2LS density matrix into the full Hilbert space by tensoring with the initial state of the oscillator
Tracer=np.kron(np.eye(4),Idv.flatten())
Expander=np.kron(np.eye(4),rhov0.flatten()).T
Ea=Tracer@EL@Expander # single-time dynamical map for the 2LS
Ea2=Tracer@EL2@Expander # two-time dynamical map for the 2LS

T2=Ea2-Ea@Ea

# print the tensor correlation matrix at time t=2*dt
print(T2.reshape(2,2,2,2))

The plots and the corresponding code may be printed and uploaded on a pdf. Label axes correctly. The analysis of the trend may be described in up to five handwritten lines with your own words.

In the class where Javier presented the exercises, he said that we are asked to (in the case of T=0 in the environment, then nbar=0 all the time), depending on the value of g, come up with a test whether the atom is markovian or non-markovian.

We wanto to compute the norm of $T_2$ as a function of $\kappa$. So we start by developing it's expression in terms of $E_k=Tr \;e^{kdtL}Ex$
$$T_2=E_2-E_1E_1=(Tr \;e^{2dtL}Ex)-(Tr \;e^{dtL}Ex)(Tr \;e^{dtL}Ex)$$
So, if we express the expander and the tracer in terms of the python language:
    Expander=np.kron(np.eye(4),rhov0.flatten()).T
    Tracer=np.kron(np.eye(4),Idv.flatten())
Then: T2=np.kron(np.eye(4),Idv.flatten())expm(2*L*dt)
